# Error handling

## Panic

* A program panics when it encounters something so messed up that there must be a bug in the program itself. Something like:
  * Out-of-bounds array access
  * Integer division by zero
  * Calling `.unwrap()` on an `Option` that happens to be `None`
  * Assertion failure
* A panic is not a crash. It’s not undefined behavior. The behavior is well-defined!
* Macro `panic!` can trigger a panic directly

In [2]:
panic!("Critical error!");


thread '<unnamed>' panicked at src\lib.rs:6:1:
Critical error!
stack backtrace:
   0: std::panicking::begin_panic_handler
             at /rustc/1159e78c4747b02ef996e55082b704c09b970588/library\std\src\panicking.rs:697
   1: core::panicking::panic_fmt
             at /rustc/1159e78c4747b02ef996e55082b704c09b970588/library\core\src\panicking.rs:75
   2: ctx::run_user_code_2::{{closure}}
   3: _rust_try.llvm.7135721178753807917
   4: std::panic::catch_unwind
   5: run_user_code_2
   6: <unknown>
   7: <unknown>
   8: <unknown>
   9: <unknown>
  10: <unknown>
  11: <unknown>
  12: <unknown>
  13: <unknown>
  14: BaseThreadInitThunk
  15: RtlUserThreadStart
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


### Unwinding

* Panic typically proceeds as follows:
  * An error message is printed to the terminal. If you set the `RUST_BACKTRACE` environment variable, as the messages suggests, Rust will also dump the stack at this point.
  * The stack is unwound. This is a lot like C++ exception handling. Any temporary values, local variables, or arguments that the current function was using are dropped, in the reverse of the order they were created. Dropping a value simply means cleaning up after it: any `String`s or `Vec`s the program was using are freed, any open `File`s are closed, and so on. User-defined `drop` methods are called too.
  * Finally, the thread exits. If the panicking thread was the main thread, then the whole process exits (with a nonzero exit code).

### Aborting

* Stack unwinding is the default panic behavior, but there are two circumstances in which Rust does not try to unwind the stack.
  * If a `.drop()` method triggers a second panic while Rust is still trying to clean up after the first, this is considered fatal. Rust stops unwinding and aborts the whole process.
  * Rust’s panic behavior is customizable. If you compile with `-C panic=abort`, the first panic in your program immediately aborts the process. 

## Result

* A `Result` represents an operation that can either succeed (returning a result) or fail (returning an error).
* It is defined as:

In [3]:
mod explain {
    pub enum Result<T, E> {
        Ok(T),
        Err(E),
    }
}

In [4]:
#[derive(Debug)]
struct ReadFileError {
    message: String
}

impl std::fmt::Display for ReadFileError {
    fn fmt(&self, f: &mut std::fmt::Formatter<'_>) -> std::fmt::Result {
        write!(f, "ReadFileError: {}", self.message)
    }
}

impl std::error::Error for ReadFileError {}

In [5]:
fn read_file(path: &str) -> Result<String, ReadFileError> {  
    let file: io::Result<String> = std::fs::read_to_string(path);
    
    match file {
        Ok(content) => Result::Ok(content),
        Err(err) => Result::Err(ReadFileError{message: err.to_string()}),
    }
}

Error: failed to resolve: use of unresolved module or unlinked crate `io`

In [6]:
let file_content = read_file("file.txt");

match file_content {
    Ok(content) => println!("{}", content),
    Err(err) => println!("{}", err),
}

Error: cannot find function `read_file` in this scope

### Result methods

* `result.unwrap()` also returns the success value, if result is a success result. However, if result is an error result, this method panics.
* `result.expect(message)` is the same as `.unwrap()`, but lets you provide a message that it prints in case of panic.
* `result.is_ok()` and `result.is_err()` return a `bool` telling if result is a success result or an error result.
* `result.ok()` returns the success value, if any, as an `Option<T>`. If result is a success result, this returns `Some(success_value)`; otherwise, it returns `None`, discarding the error value.
* `result.err()` returns the error value, if any, as an `Option<E>`.
* `result.unwrap_or(fallback)` returns the success value, if result is a success result. Otherwise, it returns `fallback`, discarding the error value.

In [7]:
static DEFAULT_FILE_CONTENT: &str = "Default content";

let file_content = read_file("unknown_file.txt").unwrap_or(DEFAULT_FILE_CONTENT.to_string());

file_content

Error: cannot find function `read_file` in this scope

* `result.unwrap_or_else(fallback_fn)` is the same, but instead of passing a fallback value directly, you pass a function or closure. This is for cases where it would be wasteful to compute a fallback value if you’re not going to use it. The `fallback_fn` is called only if we have an error result.
* `result.as_ref()` converts a `Result<T, E>` to a `Result<&T, &E>`, borrowing a reference to the success or error value in the existing result.
* `result.as_mut()` is the same, but borrows a mutable reference. The return type is `Result<&mut T, &mut E>`.

#### Mapping

* Mapping means transforming the success or error value inside a `Result`, while leaving the other value untouched. It simplifies code that would otherwise require matching on the `Result` and handling both cases.
  * `result.map(success_fn)` transforms a `Result<T, E>` to a `Result<U, E>` by applying the function `success_fn` to the success value, if any. The error value is left untouched.
  * `result.map_err(error_fn)` is the same, but applies `error_fn` to the error value, if any, transforming a `Result<T, E>` to a `Result<T, F>`.

In [10]:
fn read_file(path: &str) -> Result<String, ReadFileError> {  
    std::fs::read_to_string(path).map_err(|err| ReadFileError{message: err.to_string()})
}

let file_content = read_file("file.txt");

match file_content {
    Ok(content) => println!("{}", content),
    Err(err) => println!("{}", err),
}

This is a file with some text.


()

### Result Type Aliases

* Modules often define a `Result` type alias to avoid having to repeat an error type that’s used consistently by almost every function in the module

In [11]:
mod custom {
    pub type Result<T> = std::result::Result<T, ReadFileError>;

    #[derive(Debug)]
    pub struct ReadFileError {
        message: String
    }

    pub fn read_file(path: &str) -> Result<String> {  
        let file = std::fs::read_to_string(path);
    
        match file {
            Ok(content) => Result::Ok(content),
            Err(err) => Result::Err(ReadFileError{message: err.to_string()}),
        }
    }
}

custom::read_file("file.txt").ok()

Some("This is a file with some text.")

### Trait std::error::Error - Printing Errors

* All of standard library errors have a common interface, the `std::error::Error` trait, which means they share the following features:
  * They’re all printable using `println!()`. Printing an error with the `{}` format specifier typically displays only a brief error message. Alternatively, you can print with the `{:?}` format specifier, to get a `Debug` view of the error
  * `err.description()` returns an error message as a `&str`.
  * `err.source()` returns an `Option<&(dyn Error + 'static)>`: the underlying error, if any, that triggered `err`.

### Propagating Error

* In most places where we try something that could fail, we don’t want to catch and handle the error immediately
* Instead, if an error occurs, we usually want to let our caller deal with it. We want errors to propagate up the call stack.
* Rust has a `?` operator that does this. You can add a `?` to any expression that produces a `Result`, such as the result of a function call:

In [12]:
use std::io;

fn read_file_content(name: &str) -> io::Result<String> {
    use std::fs::File;
    use std::io::prelude::*;
 
    let mut file = File::open(name)?;     
    let mut contents = String::new();
    file.read_to_string(&mut contents)?;
    Ok(contents)
}

In [13]:
let file_content = read_file_content("unknown_file.txt");

match file_content {
    Ok(content) => println!("{}", content),
    Err(err) => println!("Error: {:?}", err),
}

Error: Os { code: 2, kind: NotFound, message: "Nie można odnaleźć określonego pliku." }


()

* The behavior of `?` depends on whether this function returns a success result or an error result:
  * on success, it unwraps the Result to get the success value inside.
  * on error, it immediately returns from the enclosing function, passing the error result up the call chain. To ensure that this works, `?` can only be used in functions that have a `Result` return type.

### Working with Multiple Error Types

In [14]:
use std::io::{self, BufRead};

fn read_numbers(file: &mut dyn BufRead) -> Result<Vec<i32>, io::Error> {
    let mut numbers = Vec::new();
    for line_result in file.lines() {
        let line = line_result?;
        let number = line.parse::<i32>()?;
        numbers.push(number);
    }
    Ok(numbers)
}

Error: `?` couldn't convert the error to `std::io::Error`

#### Case 1 - Trait `From`

In [17]:
use std::io::BufRead;

#[derive(Debug)]
enum ReadNumbersError {
    Io(io::Error),
    ParseInt(std::num::ParseIntError),
}

impl From<io::Error> for ReadNumbersError {
    fn from(err: io::Error) -> Self {
        ReadNumbersError::Io(err)
    }
}

impl From<std::num::ParseIntError> for ReadNumbersError {
    fn from(err: std::num::ParseIntError) -> Self {
        ReadNumbersError::ParseInt(err)
    }
}

fn read_numbers(file: &mut dyn BufRead) -> Result<Vec<i32>, ReadNumbersError> {
    let mut numbers = Vec::new();
    for line_result in file.lines() {
        let line = line_result?;
        let number = line.parse::<i32>()?;
        numbers.push(number);
    }
    Ok(numbers)
}

In [18]:
use std::io::{self, BufReader};

fn main() {
    let file = std::fs::File::open("numbers.txt").unwrap();

    let mut buffer_reader = BufReader::new(file);

    let numbers_result = read_numbers(&mut buffer_reader);

    match numbers_result {
        Ok(numbers) => println!("{:?}", numbers),
        Err(e) => println!("Error reading numbers: {:?}", e),
    }
}

main();

[1, 2, 3, 42, 665, 6, 7, 8]


#### Case 2 - `Box<dyn std::error::Error>`

* Code does not compile - there is no conversion from `ParseIntError` to `io::Error`
* The easiest way to handle multiple error types is to define these type aliases:

In [19]:
type GenericError = Box<dyn std::error::Error>;
type GenericResult<T> = Result<T, GenericError>;

* and change return type from function to `GenericResult<Vec<i32>>`

In [20]:
use std::io::{self, BufRead};

fn parse_numbers(file: &mut dyn BufRead) -> GenericResult<Vec<i32>> {
    let mut numbers = Vec::new();
    for line_result in file.lines() {
        let line = line_result?;
        let number = line.parse::<i32>()?;
        numbers.push(number);
    }
    Ok(numbers)
}

In [21]:
use std::io::{self, BufReader};

fn main() {
    let file = std::fs::File::open("invalid_numbers.txt").unwrap();
    let mut buffer_reader = BufReader::new(file);
    
    let numbers_result = parse_numbers(&mut buffer_reader);

    match numbers_result {
        Ok(numbers) => println!("{:?}", numbers),
        Err(e) => println!("Error reading numbers: {:?}", e),
    }
}
main();

Error reading numbers: ParseIntError { kind: InvalidDigit }


### Declaring a Custom Error Types

In [22]:
#[derive(Debug, Clone)]
pub struct JsonError {
    pub message: String,
    pub line: usize,
    pub column: usize,
}

In [23]:
use std;
use std::fmt;

// Errors should be printable.
impl fmt::Display for JsonError {
    fn fmt(&self, f: &mut fmt::Formatter) -> Result<(), fmt::Error> {
        write!(f, "{} ({}:{})", self.message, self.line, self.column)
    }
}

// Errors should implement the std::error::Error trait.
impl std::error::Error for JsonError {
    fn description(&self) -> &str {
        &self.message
    }
}

In [24]:
:dep serde_json = "1.0.64"
:dep serde = { version="1.0.127", features=["derive"] }

use serde::{Deserialize, Serialize};

#[derive(Serialize, Deserialize, Debug)]
struct Person {
    name: String,
    age: i32,
}

fn load_from_json() -> Result<Person, JsonError> {
    let json = r#"
        {
            "name": "John",
            "age": 30
        }
    "#;

    let person: Person = match serde_json::from_str(json) {
        Ok(data) => data,
        Err(err) => {
            let line = err.line();
            let column = err.column();
            let message = err.to_string();
            return Err(JsonError { message, line, column });
        }
    };

    Ok(person)
}

In [14]:
let person = load_from_json().err();
person

None

# Crates for Error Handling

## thiserror

* The `thiserror` crate provides a convenient way to define custom error types in Rust by using procedural macros. It simplifies the process of implementing the `std::error::Error` trait and provides useful features for error handling.
* With `thiserror`, you can define your error types as enums or structs and annotate them with the `#[derive(Error)]` attribute. This automatically generates the necessary implementations for the `Error` trait, including methods like `source()` and `description()`.
* The crate also allows you to specify error messages using the `#[error("...")]` attribute, which supports formatting and interpolation of fields within the error type.
* Additionally, `thiserror` supports automatic conversion from other error types using the `#[from]` attribute, making it easy to wrap and propagate errors from different sources.

In [25]:
:dep thiserror = "2.0"

mod thiserror_example {
    use thiserror::Error;

    #[derive(Error, Debug)]
    pub enum ParseNumberError {
        #[error("File not found")]
        NotFound(#[from] std::io::Error),        // automatic conversion from std::io::Error
        #[error("Failed to parse number: {0}")]
        ParseError(String),
    }

    pub fn parse_numbers(path: &str) -> Result<Vec<i32>, ParseNumberError> {
        use std::fs::File;
        use std::io::{self, BufRead, BufReader};

        let file = File::open(path)?;
        let reader = BufReader::new(file);
        let mut numbers = Vec::new();

        for line_result in reader.lines() {
            let line = line_result?;
            match line.parse::<i32>() {
                Ok(num) => numbers.push(num),
                Err(_) => return Err(ParseNumberError::ParseError(line)),
            }
        }

        Ok(numbers)
    }
}

fn main() {
    match thiserror_example::parse_numbers("invalid_numbers.txt") {
        Ok(numbers) => println!("Parsed numbers: {:?}", numbers),
        Err(e) => println!("Error parsing numbers: {}", e),
    }
}

main();

Error parsing numbers: Failed to parse number: 66d5


## anyhow

* The `anyhow` crate provides a simple and flexible way to handle errors in Rust applications. It allows you to create and propagate errors without having to define custom error types for every possible error scenario.
* With `anyhow`, you can use the `anyhow::Error` type to represent errors, which can encapsulate any error type that implements the `std::error::Error` trait. This makes it easy to work with different error types without having to define a separate error enum or struct for each case.
* The crate also provides convenient macros like `anyhow!` for creating errors with formatted messages, and the `Context` trait for adding additional context to errors as they propagate through your code.
* Library convinent when prototyping or building applications where you want to focus on error handling without the overhead of defining numerous custom error types.

In [26]:
:dep anyhow = "1.0"

mod anyhow_example {
    use anyhow::Context;
    use std::io::{BufRead, BufReader};

    pub fn parse_numbers(file: &mut dyn BufRead) -> anyhow::Result<Vec<i32>> {
        let mut numbers = Vec::new();
        for (index, line_result) in file.lines().enumerate() {
            let line = line_result.context(format!("Failed to read line {}", index + 1))?;
            let number = line.parse::<i32>()
                .context(format!("Failed to parse '{}' as i32 on line {}", line, index + 1))?;
            numbers.push(number);
        }
        Ok(numbers)
    }

    pub fn read_numbers(path: &str) -> anyhow::Result<Vec<i32>> {
        let file = std::fs::File::open(path)
            .with_context(|| format!("Failed to open file '{}'", path))?;
        
        let mut buffer_reader = BufReader::new(file);
        let numbers = parse_numbers(&mut buffer_reader)
            .with_context(|| format!("Failed to read numbers from file '{}'", path))?;
        
        Ok(numbers)
    }
}

fn main() {
    let result = anyhow_example::read_numbers("numbers.txt");   
    match result {
        Ok(numbers) => println!("{:?}", numbers),
        Err(err) => println!("Error: {:?}", err),
    }
}

main();

[1, 2, 3, 42, 665, 6, 7, 8]


In [27]:
use anyhow::{anyhow, Result};

fn get_user_age(name: &str) -> Result<u32> {
    match name {
        "Alice" => Ok(30),
        "Bob" => Ok(25),
        _ => Err(anyhow!("No age data available for user '{}'", name)),
    }
}

fn main() -> Result<()> {
    let age = get_user_age("Charlie")?;
    println!("User age: {}", age);
    Ok(())
}

main();